In [1]:
from MAUtil import *
from MAInit import *
import time

In [2]:
#############################################################################3
# name = 'Au_111_u2'
# name = 'Au_100_u2'
# name = 'Pd_111_u2'
# name = 'Pd_100_u2'
# name = 'Al_111_u2'
# name = 'Al_100_u2'
# name = 'Ni_111_u2'
# name = 'Ni_100_u1'
# name = 'Ru_0001_u2'
name = 'Zn_0001_u2'
# name = 'Rh_111_u2'
# name = 'Rh_100_u2'
name = 'Ag_100_u2_no03_CO_n3_d3'
name = 'Cu_100_u2_RPBE'


inname = name + '.traj'
atoms = init_query(inname, 'spacom')
baresurface, initadsites = remove_molecule(atoms, ['C', 'O'])

In [3]:
bareadsites = getadsites(baresurface, False)
d=1.148

maxmole = 4
mindist = 0.000001 # must be not 0
molecule = Atoms('CO', positions=[(0., 0., 0.), (0., 0., d)], cell=(10., 10., 10.))

In [4]:
allbareadsites = np.array(bareadsites['all'])
group = create_site_group(baresurface)

ind = []
for i in range(len(allbareadsites)):
    for j in range(len(initadsites)):
        if np.allclose(allbareadsites[i][:2], initadsites[j][:2]):
            ind.append(i)

adsites = np.delete(allbareadsites,ind,0)
inuse = allbareadsites[ind]
# adsites, inuse

In [ ]:
start = time.time()

allatoms, allused, mindistlis, numdict, molenum \
    = getuniqueatoms(atoms, baresurface, adsites, maxmole, mindist, initadsites, group, molecule)
print(time.time() - start)

progress: 1/3


In [ ]:
print(len(allatoms))

for num in numdict:
    print(num, numdict[num])

In [ ]:
mindistlis

In [ ]:
count = 0
indexlist = []
for i in range(len(mindistlis)):
    if mindistlis[i] > 2.5:
        indexlist.append(i)
        count += 1
# count, indexlist

In [ ]:
# for i in range(len(allatoms)): # len(allatoms)
# #     if 'COCOCOCOCO' in str(allatoms[i].symbols):
#     view(allatoms[i])
#     print(allatoms[i].symbols)
# #         print('a')

In [ ]:
for i in range(len(allatoms)):
#     if 'COCOCOCOCO' in str(allatoms[i].symbols):
    outname = name + str('_no') + str('{0:03d}'.format(i+1)) +'_CO_n' + str(molenum[i]) + str('_d') + str(int(np.ceil(mindistlis[i]/0.5)-3))  + '.traj'
    print(outname)
    outpath = '/home/katsuyut/init/' + str(outname)
    allatoms[i].write(outpath)